# Managing conversation history with the ChatGPT model
This sample notebook demonstrates a couple of simple patterns you can use for managing the prompts and conversation history with the ChatGPT model.

> if needed, install and/or upgrade to the latest version of the OpenAI Python library

In [1]:
!pip install --upgrade openai
!pip install python-dotenv
!pip install tiktoken

In [2]:
# import os module & the OpenAI Python library for calling the OpenAI API
# please make sure you have installed required libraries via pip install -r requirements.txt
import os, dotenv
import openai

dotenv.load_dotenv()

True

In [3]:
# Setting up the deployment name
deployment_name = "gpt-4o-mini" #FIXME

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
# This is the value of the endpoint for your Azure OpenAI resource
azure_endpoint = os.environ['AZURE_OPENAI_ENDPOINT']

# The API key for your Azure OpenAI resource.
api_key = os.getenv("AZURE_OPENAI_API_KEY") #FIXME

# Currently OPENAI API have the following versions available: https://learn.microsoft.com/azure/ai-services/openai/reference
api_version = "2024-07-01-preview"

client = openai.AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=azure_endpoint
) #FIXME

## 1.0 Create the system message for ChatGPT

In [4]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"
print(system_message)

You are a helpful assistant.


## 2.0 Define helper functions



In [5]:
import tiktoken

def num_tokens_from_messages(messages, model="gpt-4o-mini"):
    encoding = tiktoken.encoding_for_model(model) #FIXME
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens #FIXME

In [6]:
# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(messages, model_name, max_response_tokens=500):
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        max_tokens=max_response_tokens
    ) #FIXME
    return response.choices[0].message.content #FIXME

# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message['content'])
        print()

## 3.0 Start the conversation

In [7]:
# This is the first user message that will be sent to the model. Feel free to update this.
user_message = "I want to write a blog post about the impact of AI on the future of work."

In [8]:
# Create the list of messages. role can be either "user" or "assistant" 
messages= [{"role":"system", "content": f"{system_message}"},
           {"role":"user", "content": f"{user_message}"}] #FIXME

In [9]:
token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

Token count: 36


In [10]:
max_response_tokens = 100

response = send_message(messages, deployment_name, max_response_tokens)
messages.append({"role": "assistant", "content": response})

print_conversation(messages)

[SYSTEM]
You are a helpful assistant.

[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
That sounds like a fascinating topic! The impact of AI on the future of work is a rich area to explore, and there are several key points and themes you could cover. Below is an outline and some ideas to help you organize your blog post:

### Blog Post Outline: The Impact of AI on the Future of Work

#### Introduction
- Brief overview of AI and its rapid development.
- Importance of discussing its implications for the workforce.
- Thesis statement: AI is transforming the workplace, altering job



## 4.0 Continue the conversation

When working with the ChatGPT model, it's your responsibity to make sure you stay within the token limits of the model. The model can handle a maximum of 4096 tokens, and this includes the number of tokens in the prompt as well as the `max_tokens` you're requesting from the model. If you exceed these limits, the model will return an error.

You should also consider the trade-off between maintaining more of the conversation history and the cost/latency that you'll incur by including those tokens in the prompt. Shorter prompts are cheaper and faster. The amount of the previous conversation you include also makes a difference in how the model responds.

In this notebook, we'll show two strategies for managing the conversation history when working with the ChatGPT model.
- Option 1: Keep the conversation within a given token limit
- Option 2: Keep the conversation within a given number of turns

### Option 1: Keep the conversation within a given token limit

`overall_max_tokens` is the maximum number of tokens that you want to include in the prompt. Th maximum number this can be set to is 4096 but you can also consider reducing this number to reduce the cost and latency of the request.

In [11]:
overall_max_tokens = 1096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [12]:
user_message = "The target audience for the blog post should be business leaders working in the tech industry."
#user_message = "Let's talk about generative AI and keep the tone informational but also friendly."
#user_message = "Show me a few more examples"
messages.append({"role": "user", "content": user_message})

token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

# remove first message while over the token limit
while token_count > prompt_max_tokens:
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = send_message(messages, deployment_name, max_response_tokens) #FIXME

#Append assistance response 
messages.append({"role": "assistant", "content": response}) #FIXME
print_conversation(messages)

Token count: 163
[SYSTEM]
You are a helpful assistant.

[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
That sounds like a fascinating topic! The impact of AI on the future of work is a rich area to explore, and there are several key points and themes you could cover. Below is an outline and some ideas to help you organize your blog post:

### Blog Post Outline: The Impact of AI on the Future of Work

#### Introduction
- Brief overview of AI and its rapid development.
- Importance of discussing its implications for the workforce.
- Thesis statement: AI is transforming the workplace, altering job

[USER]
The target audience for the blog post should be business leaders working in the tech industry.

[ASSISTANT]
Great! Tailoring your blog post for business leaders in the tech industry means you can incorporate insights that are relevant to their strategic decision-making. Here's a refined outline and some content ideas that focus on the implic

### Option 2: Keep the conversation within a given number of turns

In [13]:
max_messages = 10

overall_max_tokens = 4096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [14]:
user_message = "Make the post about generative AI aimed at business leaders who have some knowledge of the technology."
messages.append({"role": "user", "content": user_message})

while len(messages) > max_messages:
    messages.pop(0)

token_count = num_tokens_from_messages(messages)

while token_count > prompt_max_tokens:
    # remove first message from messages
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = send_message(messages, deployment_name, max_response_tokens)#FIXME

#Append assistant response
messages.append({"role": "assistant", "content": response}) #FIXME
print_conversation(messages)


[SYSTEM]
You are a helpful assistant.

[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
That sounds like a fascinating topic! The impact of AI on the future of work is a rich area to explore, and there are several key points and themes you could cover. Below is an outline and some ideas to help you organize your blog post:

### Blog Post Outline: The Impact of AI on the Future of Work

#### Introduction
- Brief overview of AI and its rapid development.
- Importance of discussing its implications for the workforce.
- Thesis statement: AI is transforming the workplace, altering job

[USER]
The target audience for the blog post should be business leaders working in the tech industry.

[ASSISTANT]
Great! Tailoring your blog post for business leaders in the tech industry means you can incorporate insights that are relevant to their strategic decision-making. Here's a refined outline and some content ideas that focus on the implications of AI for 

## Ejercicio 1: Modificar el Mensaje del Sistema
Enunciado:
Crea un mensaje del sistema que defina a ChatGPT como un "analista financiero experto". Imprime el mensaje del sistema resultante y utilízalo en un mensaje inicial del usuario que pregunte:
"¿Cuáles son las tendencias actuales en el mercado de acciones tecnológicas?".


In [15]:

# Mensaje del sistema
base_system_message = "You are a financial analyst expert."
system_message = f"{base_system_message.strip()}"
print(system_message)

# Crear lista de mensajes
user_message = "What are the current trends in the tech stock market?"
messages = [{"role": "system", "content": system_message},
            {"role": "user", "content": user_message}] #FIXME

# Enviar mensaje y recibir respuesta
response = send_message(messages, deployment_name, max_response_tokens) #FIXME

messages.append({"role": "assistant", "content": response})
print_conversation(messages)


You are a financial analyst expert.
[SYSTEM]
You are a financial analyst expert.

[USER]
What are the current trends in the tech stock market?

[ASSISTANT]
As of my last update in October 2023, several trends were influencing the tech stock market. Here are some notable ones:

1. **Artificial Intelligence (AI) Boom**: The rapid advancement and adoption of AI technologies have significantly boosted valuations for companies in AI and machine learning. Firms specializing in AI tools, software, and services were attracting substantial investor interest.

2. **Continued Focus on Cloud Computing**: Cloud service providers continued to report strong earnings and growth, spurred by digital transformation



## Ejercicio 2: Contar Tokens en una Conversación Extendida
Enunciado:
Dado un conjunto inicial de mensajes, agrega tres mensajes adicionales:

"Explain the impact of inflation on stock prices."
"What are the top stocks to watch in 2025?"
"Summarize the latest market news in 50 words."
Calcula y muestra el número total de tokens tras añadir cada mensaje.


In [17]:

# Mensajes iniciales
messages = [
    {"role": "system", "content": "You are a helpful assistant specialized in financial topics."},
    {"role": "user", "content": "What are the current trends in the tech stock market?"}
]

# Mensajes adicionales
additional_messages = [
    "Explain the impact of inflation on stock prices.",
    "What are the top stocks to watch in 2025?",
    "Summarize the latest market news in 50 words."
]

# Agregar mensajes y calcular tokens
for msg in additional_messages:
    messages.append({"role": "user", "content": msg})
    token_count = num_tokens_from_messages(messages) #FIXME
    print(f"Token count after adding '{msg}': {token_count}")


Token count after adding 'Explain the impact of inflation on stock prices.': 47
Token count after adding 'What are the top stocks to watch in 2025?': 64
Token count after adding 'Summarize the latest market news in 50 words.': 81


## Ejercicio 3: Personalizar el Mensaje del Sistema Según el Usuario
Enunciado:
Escribe un programa que permita al usuario definir el rol de ChatGPT mediante una entrada de texto (input). Una vez definido, responde al mensaje:
"What are your key skills as an assistant?"
Usa el mensaje del sistema definido por el usuario.


In [20]:

# Entrada del usuario para definir el mensaje del sistema
user_defined_role = input("Define the role of the assistant: ")
system_message = f"You are {user_defined_role.strip()}."

# Crear lista de mensajes
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "What are your key skills as an assistant?"}
]

# Enviar mensaje y recibir respuesta
response = send_message(messages, deployment_name, max_response_tokens) #FIXME

messages.append({"role": "assistant", "content": response})

# Mostrar la conversación
print_conversation(messages)


[SYSTEM]
You are a helpful assistant.

[USER]
What are your key skills as an assistant?

[ASSISTANT]
As an assistant, my key skills include:

1. **Information Retrieval**: I can quickly provide accurate information on a wide range of topics, from factual data to general knowledge.

2. **Problem-Solving**: I can assist in troubleshooting issues and suggest potential solutions based on the information provided.

3. **Natural Language Processing**: I understand and interpret user inquiries in natural language, making interactions intuitive and conversational.

4. **Research Assistance**: I can help with gathering information, summar



## Ejercicio 4: Crear un Generador de Respuestas Creativo
Enunciado:
Diseña un programa que permita configurar la creatividad del modelo (temperatura) a través de una entrada del usuario. Una vez configurado, responde a la pregunta:
"Can you suggest an out-of-the-box idea for a startup?"



In [23]:

# Entrada del usuario para definir la creatividad
temperature = float(input("Set the creativity level (0.0 - 1.0): "))
if not (0.0 <= temperature <= 1.0):
    raise ValueError("Creativity level must be between 0.0 and 1.0.")

# Crear lista de mensajes
system_message = "You are a highly creative assistant."
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "Can you suggest an out-of-the-box idea for a startup?"}
]

# Enviar mensaje con creatividad ajustada
response = client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    max_tokens=max_response_tokens,
    temperature=temperature
) #FIXME

# Mostrar la respuesta
print(temperature)
print(response.choices[0].message.content)

0.7
Absolutely! Here’s an innovative startup idea that blends technology, sustainability, and community engagement:

**Startup Idea: "EcoSwap" - A Localized Circular Economy Platform**

### Concept:
EcoSwap is a mobile application and website that facilitates the exchange of goods and services within local communities to promote sustainability and reduce waste. The platform allows users to swap items they no longer need (like clothing, furniture, electronics) or offer services (like tutoring, gardening, or handyman work) for goods or services they


## Ejercicio 5: Cambiar el Tono del Mensaje
Enunciado:
Configura el mensaje del sistema para que ChatGPT responda en un tono humorístico y escribe una respuesta al mensaje:
"Explain quantum computing in a funny way."


In [25]:

# Mensaje del sistema humorístico
system_message = "You are a humorous assistant."
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "Explain quantum computing in a funny way."}
]

# Enviar mensaje y recibir respuesta
response = send_message(messages, deployment_name, max_response_tokens) #FIXME

#append assistant response
messages.append({"role": "assistant", "content": response}) #FIXME
print_conversation(messages)


[SYSTEM]
You are a humorous assistant.

[USER]
Explain quantum computing in a funny way.

[ASSISTANT]
Alright, imagine you’re at a party with some friends and you’re all trying to find out who has the best dance moves. 

In the classic computing corner, you’ve got a group of party-goers who can only dance one at a time. So, they take turns hitting the dance floor, and the whole process is slower than a sloth in a hammock. One declares, "I can do the robot!" but you have to wait till they finish before you find out if their robot



## Ejercicio 6: Resumir Mensajes Previos
Enunciado:
Dado un historial de conversación de 5 mensajes, crea una función que resuma el contenido de los mensajes del usuario en una sola oración.



In [26]:

# Historial de mensajes
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Tell me about AI."},
    {"role": "user", "content": "How does machine learning work?"},
    {"role": "user", "content": "Can you explain neural networks?"},
    {"role": "user", "content": "What are the applications of deep learning?"}
]

# Resumir mensajes del usuario
def summarize_user_messages(messages):
    user_messages = [msg['content'] for msg in messages if msg['role'] == 'user']
    summary_prompt = "Summarize the following messages into one sentence: " + " ".join(user_messages)
    summary = send_message([{"role": "system", "content": "You are a helpful assistant."},
                            {"role": "user", "content": summary_prompt}], deployment_name)
    return summary

# Obtener y mostrar el resumen
summary = summarize_user_messages(messages)
print(f"User messages summary: {summary}")


User messages summary: AI involves the simulation of human intelligence in machines, where machine learning utilizes algorithms to enable systems to learn from data; neural networks, a subset of machine learning, mimic the human brain's structure to recognize patterns, while deep learning, leveraging complex neural networks, is applied in various fields such as image and speech recognition, natural language processing, and autonomous systems.


## Ejercicio 7: Configuración de Temperatura y Tono
Enunciado:
Escribe un programa que cambie la configuración de temperatura a 0.9 para obtener respuestas más creativas y responde al mensaje:
"Suggest a unique title for a blog about AI in education."



In [27]:

# Mensajes iniciales
messages = [
    {"role": "system", "content": "You are a creative assistant."},
    {"role": "user", "content": "Suggest a unique title for a blog about AI in education."}
]

# Ajustar temperatura y enviar mensaje
response = client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    max_tokens=max_response_tokens,
    temperature=temperature
) #FIXME

# Mostrar la respuesta
print(response.choices[0].message.content)


"Learning Reimagined: The AI Classroom Revolution"
